# Streaming Prediction with Spark ML and Spark Streaming

In this notebook we are going to train a classification model to predict a patient's probability of suffering a heart attack.

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.types import *

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('UCI Heart disease').getOrCreate()

In [ ]:
heart = spark.read.csv('data/heart.csv', 
                       inferSchema = True, 
                       header = True)
heart.show(3)

+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
|age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
| 63|  1|  3|     145| 233|  1|      0|    150|    0|    2.3|    0|  0|   1|     1|
| 37|  1|  2|     130| 250|  0|      1|    187|    0|    3.5|    0|  0|   2|     1|
| 41|  0|  1|     130| 204|  0|      0|    172|    0|    1.4|    2|  0|   2|     1|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
only showing top 3 rows



In [ ]:
schema = StructType( \
                     [StructField("age", LongType(),True), \
                      StructField("sex", LongType(), True), \
                      StructField("cp", LongType(), True), \
                      StructField('trestbps', LongType(), True), \
                      StructField("chol", LongType(), True), \
                      StructField("fbs", LongType(), True), \
                      StructField("restecg", LongType(), True), \
                      StructField("thalach", LongType(), True),\
                      StructField("exang", LongType(), True), \
                      StructField("oldpeak", DoubleType(), True), \
                      StructField("slope", LongType(),True), \
                      StructField("ca", LongType(), True), \
                      StructField("thal", LongType(), True), \
                      StructField("target", LongType(), True), \
                        ])

In [ ]:
from pyspark.ml import Pipeline
from pyspark.sql.types import StructType,StructField,LongType, StringType,DoubleType,TimestampType

df = heart.withColumnRenamed("target","label")
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- sex: integer (nullable = true)
 |-- cp: integer (nullable = true)
 |-- trestbps: integer (nullable = true)
 |-- chol: integer (nullable = true)
 |-- fbs: integer (nullable = true)
 |-- restecg: integer (nullable = true)
 |-- thalach: integer (nullable = true)
 |-- exang: integer (nullable = true)
 |-- oldpeak: double (nullable = true)
 |-- slope: integer (nullable = true)
 |-- ca: integer (nullable = true)
 |-- thal: integer (nullable = true)
 |-- label: integer (nullable = true)



In [ ]:
testDF, trainDF = df.randomSplit([0.3, 0.7])

In [ ]:
# Create the logistic regression model
lr = LogisticRegression(maxIter=10, regParam= 0.01)

In [ ]:
# We create a one hot encoder.
ohe = OneHotEncoder(inputCols = ['sex', 'cp', 'fbs', 'restecg', 'slope', 
                                 'exang', 'ca', 'thal'], 
                    outputCols=['sex_ohe', 'cp_ohe', 'fbs_ohe', 
                                'restecg_ohe', 'slp_ohe', 'exng_ohe', 
                                'caa_ohe', 'thall_ohe'])

# Input list for scaling
inputs = ['age','trestbps','chol','thalach','oldpeak']

# We scale our inputs
assembler1 = VectorAssembler(inputCols=inputs, outputCol="features_scaled1")
scaler = MinMaxScaler(inputCol="features_scaled1", outputCol="features_scaled")

# We create a second assembler for the encoded columns.
assembler2 = VectorAssembler(inputCols=['sex_ohe', 'cp_ohe', 
                                        'fbs_ohe', 'restecg_ohe', 
                                        'slp_ohe', 'exng_ohe', 'caa_ohe', 
                                        'thall_ohe','features_scaled'], 
                             outputCol="features")


In [ ]:
# Create stages list
myStages = [assembler1, scaler, ohe, assembler2,lr]

# Set up the pipeline
pipeline = Pipeline(stages= myStages)

In [ ]:
# We fit the model using the training data.
pModel = pipeline.fit(trainDF)

# We transform the data.
trainingPred = pModel.transform(trainDF)

# # We select the actual label, probability and predictions
trainingPred.select('label','probability','prediction').show()

+-----+--------------------+----------+
|label|         probability|prediction|
+-----+--------------------+----------+
|    1|[0.02547572141032...|       1.0|
|    1|[0.00529737717862...|       1.0|
|    1|[0.01433771555471...|       1.0|
|    1|[0.03186502726837...|       1.0|
|    0|[0.91655787910351...|       0.0|
|    0|[0.76284874224638...|       0.0|
|    1|[0.02615684728437...|       1.0|
|    1|[0.00285662444432...|       1.0|
|    1|[0.03955333515046...|       1.0|
|    1|[0.03960488368718...|       1.0|
|    0|[0.55115516202987...|       0.0|
|    1|[0.00119850446011...|       1.0|
|    1|[0.01842522915928...|       1.0|
|    0|[0.70706060938886...|       0.0|
|    1|[0.03740470373666...|       1.0|
|    0|[0.94157151280181...|       0.0|
|    0|[0.30476847727903...|       1.0|
|    1|[0.19848850346177...|       1.0|
|    1|[0.04391393058917...|       1.0|
|    1|[0.01665766697008...|       1.0|
+-----+--------------------+----------+
only showing top 20 rows



In [ ]:
pModel.save("/pipelines")